# Run this first to download test data

In [ ]:
import requests

url = "https://hds5210-data.s3.amazonaws.com/flu_2016_2017.csv"
filename = "flu_2016_2017.csv"

response = requests.get(url)
with open(filename, "wb") as file:
  file.write(response.content)

url = "https://hds5210-data.s3.amazonaws.com/hhcaps.csv"
filename = "hhcaps.csv"

response = requests.get(url)
with open(filename, "wb") as file:
  file.write(response.content)

### Books on Business Graphics

Wall Street Journal Guide to Information Graphics by Dona Wong

https://www.amazon.com/Street-Journal-Guide-Information-Graphics/dp/0393347281

---

* Exploratory plots with pandas
  * Histograms
  * Scatter Plots
  
* Descriptive and presentable plots with matplotlib
  * Multiple plots per figure
  * Adjusting axes and formatting

---

Use `%matplotlib notebook` to make plots interactive

In [ ]:
%matplotlib inline

Plotting from pandas
---

In [ ]:
import pandas as pd
data = pd.read_csv('flu_2016_2017.csv', quotechar='"')
usa = data[(data['Country'] == 'United States of America') &
           (data['Year'] == 2016)]
usa.shape

In [ ]:
data[0:2]

In [ ]:
usa.dtypes

In [ ]:
usa[0:1]

In [ ]:
country_filter = data['Country'] == 'United States of America'

In [ ]:
country_filter.head()

In [ ]:
data[country_filter].head()

In [ ]:
filter_year = data['Year'] == 2016

In [ ]:
filter_year.head()

In [ ]:
(filter_year | country_filter).head()

In [ ]:
filter_big = data['SPEC_RECEIVED_NB'] > 21000

In [ ]:
data[(filter_year & country_filter & filter_big)]

In [ ]:
data.query('Country == "United States of America" & Year == 2016 & SPEC_RECEIVED_NB > 22000')

---

Histogram
---

Plot a histogram of the number of Influenza B cases per week

In [ ]:
usa

In [ ]:
usa['INF_B'].hist()

In [ ]:
usa[['INF_B','INF_A','SPEC_RECEIVED_NB']].hist(bins=5)


---

Line Plot (default)
---

Plot the number of cases by week

In [ ]:
import seaborn as sns
usa.plot(x='SDATE', y=['INF_B','INF_A','SPEC_RECEIVED_NB','SPEC_PROCESSED_NB', 'ALL_INF'])

In [ ]:
usa[0:10].plot(x='SDATE', y=['INF_B','INF_A','SPEC_RECEIVED_NB','SPEC_PROCESSED_NB'])

---

Scatter Plot
---

Use a scatter plot to see if there are some possible relationship

In [ ]:
axes = usa.plot.scatter(x='SPEC_RECEIVED_NB', y='INF_A', color='DarkRed', label='Influeza A')
axes = usa.plot.scatter(x='SPEC_RECEIVED_NB', y='INF_B', color='DarkBlue', label='Influenza B', ax=axes)
axes = usa.plot.scatter(x='SPEC_RECEIVED_NB', y='ALL_INF', color='LightGreen', label='All Flu', ax=axes)

axes.set_ylabel('Positive Cases')
axes.set_xlabel('Specimens Received')

In [ ]:
import seaborn as sns
ax = sns.regplot(data=usa, x='SPEC_RECEIVED_NB', y='INF_A', order=2, label='Flu A')
sns.regplot(data=usa, x='SPEC_RECEIVED_NB', y='INF_B', label='Flu B', ax=ax)
ax.legend()
ax.set_ylabel('Positive Cases')
ax.set_xlabel('Specimens Received')

In [ ]:
import seaborn as sns
ax = sns.jointplot(data=usa, x='SPEC_RECEIVED_NB', y='INF_B')


---

See if there are always a similar ratio of type A to type B cases of influenza

In [ ]:
ax = usa.plot.scatter(x='INF_A', y='INF_B')

Other Examples
===

Plot by State
---

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

d = pd.read_csv('hhcaps.csv', quotechar='"')
d.replace(to_replace='Not Available', value=0, inplace=True)
d = d[d['State'].isin(['CA','IL','WY'])]
by_state = d.groupby('State')

In [ ]:
d.head()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling

for name, group in by_state:
    recommend = group['Percent of patients who reported YES, they would definitely recommend the home health agency to friends and family'].astype(float)
    professional = group['Percent of patients who reported that their home health team gave care in a professional way'].astype(float)
    ax.plot(recommend, professional, marker='o', linestyle='', ms=5, alpha=0.5, label=name)
ax.legend()

plt.show()

Time Series Plots
---

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


d = pd.read_csv('hhcaps.csv', quotechar='"')
d = d[d['Percent of patients who reported YES, they would definitely recommend the home health agency to friends and family'] != 'Not Available']
#d.replace(to_replace='Not Available', value=0, inplace=True)


In [ ]:
d.dtypes

In [ ]:
d['Date Certified'].head()

In [ ]:
data = pd.DataFrame()
data['Year Certified'] = pd.to_datetime(d['Date Certified'].str[0:4] + '-01-01', format='%Y-%m-%d')
data['Percent Recommend'] = d['Percent of patients who reported YES, they would definitely recommend the home health agency to friends and family'].astype(float)


g = data.groupby(['Year Certified']).mean().plot(style='-')
g.set_ylim(0,100)
g

In [ ]:
data = pd.DataFrame()
data['Date Certified'] = pd.to_datetime(d['Date Certified'].str[0:4] + '-01-01', format='%Y-%m-%d')
data['Percent Recommend'] = d['Percent of patients who reported YES, they would definitely recommend the home health agency to friends and family'].astype(float)

g = data.groupby(['Date Certified']).mean().plot(style='.', alpha=.1)
g.set_ylim(0,100)

In [ ]:
recommend = pd.DataFrame()
recommend['Date Certified'] = pd.to_datetime(d['Date Certified'].str[0:4] + '-01-01', format='%Y-%m-%d')
recommend['Percent Recommend'] = d['Percent of patients who reported YES, they would definitely recommend the home health agency to friends and family'].astype(float)

data = recommend.groupby(['Date Certified']).mean()

ma = data.rolling(20).mean()

mstd = data.rolling(20).std()

plt.figure()

plt.plot(data.index, data, 'k')

plt.plot(ma.index, ma, 'b')

#plt.fill_between(mstd.index, ma-2*mstd, ma+2*mstd, color='b', alpha=0.2)
